In [112]:
from ImportLeague import getLeague
from ImportData import getTeamData
from ImportData import getPlayerData
import pandas as pd
# from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import Data
myLeague = getLeague()
teamData_defense,teamData_offense = getTeamData()
playerData = getPlayerData(myLeague,teamData_defense,teamData_offense)

In [48]:
def get_active_roster(Roster):

    myRoster = pd.DataFrame(columns=['Position','Name','Proj Points'])

    # Create a dataframe of the full roster of players I own
    for player in Roster:
        playerDict = {}
        playerDict['Position'] = player.position
        playerDict['Name'] = player.name
        playerDict['Proj Points'] = player.stats[myLeague.nfl_week]['projected_points']
        myRoster = myRoster.append(playerDict, ignore_index = True)

    # Concatenate the n highest projected-points players with n being the allowed slots per position accross positions
    myRoster = pd.concat([myRoster[myRoster['Position'] == 'QB'].nlargest(1, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'RB'].nlargest(2, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'WR'].nlargest(3, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'TE'].nlargest(1, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'D/ST'].nlargest(1, ['Proj Points']),\
        myRoster[myRoster['Position'] == 'K'].nlargest(1, ['Proj Points'])])

    return myRoster


In [104]:
myRoster = get_active_roster(myLeague.teams[8].roster)
myRoster = myRoster.reset_index(drop=True)
myRoster

,Position,Name,Proj Points
0,QB,Dak Prescott,22.02
1,RB,Saquon Barkley,14.37
2,RB,Chase Edmonds,14.32
3,WR,Amari Cooper,12.07
4,WR,Marvin Jones Jr.,11.71
5,WR,Jakobi Meyers,11.01
6,TE,Noah Fant,8.62
7,D/ST,Broncos D/ST,7.18
8,K,Mason Crosby,8.08


In [120]:
# Find sizes of each free agency pool
sizes = []

for position in ['QB','RB','WR','TE','D/ST','K']:
    sizes.append(playerData[position][playerData[position]['Free Agency'] == True].shape[0])

In [148]:
def get_action(epsilon,Q_val,sizes_Dict,position_Dict):
    p_epsilon = np.random.uniform(0,1)
    if p_epsilon < epsilon:
        position = np.random.randint(8)
        free_agent = np.random.randint(sizes_Dict[position_Dict[position]])
        return (position,free_agent)

    action = np.unravel_index(Q_val.argmax(), Q_val.shape)
    while action[1] > (sizes_Dict[position_Dict[action[0]]] -1):
        Q_val[action[0],action[1]] = -100
        action = np.unravel_index(Q_val.argmax(), Q_val.shape)



    return action


In [149]:
NUM_EPISODES = 10000
EPSILON = 0.3

# Sets Q-value table and vistis table to calculate inctemental averages
Q_val = np.zeros([8,max(sizes)])
n_s = np.zeros([8,max(sizes)])
sizes_Dict = dict(zip(['QB','RB','WR','TE','D/ST','K'], sizes))
position_Dict = dict(zip(range(8),['QB','RB','RB','WR','WR','WR','TE','D/ST','K']))
Baseline = myRoster['Proj Points'].sum()

for episode in tqdm(range(NUM_EPISODES)):

    newRoster = myRoster.copy()

    action = get_action(EPSILON,Q_val,sizes_Dict,position_Dict)

    position = position_Dict[action[0]]
    newRoster['Name'][action[0]] = playerData[position][playerData[position]['Free Agency'] == True].iloc[action[1]]['Name']
    newRoster['Proj Points'][action[0]] = playerData[position][playerData[position]['Free Agency'] == True].iloc[action[1]]['Projected Points']

    reward = newRoster['Proj Points'].sum() - Baseline

    n_s[action[0],action[1]] += 1
    Q_val[action[0],action[1]] = Q_val[action[0],action[1]] + 1/n_s[action[0],action[1]] * (reward- Q_val[action[0],action[1]])


100%|██████████| 10000/10000 [00:19<00:00, 512.22it/s]


In [153]:
Q_val

array([[  -4.72,   -7.21,   -2.39,   -7.46,   -3.09,   -6.71,   -8.8 ,
          -6.54, -100.  ],
       [  -8.19,   -9.78,   -7.16,   -9.76,  -10.28, -100.  , -100.  ,
        -100.  , -100.  ],
       [  -8.14,   -9.73,   -7.11,   -9.71,  -10.23, -100.  , -100.  ,
        -100.  , -100.  ],
       [  -4.4 ,   -5.74,   -3.72,   -5.05,   -4.02,   -6.49,   -4.55,
          -6.03,   -6.68],
       [  -4.04,   -5.38,   -3.36,   -4.69,   -3.66,   -6.13,   -4.19,
          -5.67,   -6.32],
       [  -3.34,   -4.68,   -2.66,   -3.99,   -2.96,   -5.43,   -3.49,
          -4.97,   -5.62],
       [  -2.11,   -3.17,   -5.81,   -3.63, -100.  , -100.  , -100.  ,
        -100.  , -100.  ],
       [  -1.33,   -1.07,   -2.1 ,   -3.87,   -1.38,   -1.79, -100.  ,
        -100.  , -100.  ]])